# Debug SqueezeNet v1.3 (Simple Task) OpenCL implement with PyOpenCL and PyTorch
Partial code are copied heavily from https://github.com/pytorch/vision/blob/master/torchvision/models/squeezenet.py  
SqueezeNet Paper:https://arxiv.org/abs/1602.07360  
SqueezeNet 1.1 model from https://github.com/DeepScale/SqueezeNet/tree/master/SqueezeNet_v1.1   
SqueezeNet 1.1 has 2.4x less computation and slightly fewer parameters than SqueezeNet 1.0, without sacrificing accuracy.

TEST DE IMPLEMENTACIÓN CONV1x1

In [1]:
#some set up
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
import math
import time
from time import sleep, perf_counter as pc
from matplotlib.pyplot import imshow
%matplotlib inline

## conv1x1 con opencl Simple task Vs NDRange

In [2]:
# OpenCL setup
import pyopencl as cl
import sys
sys.path.append('../python_common')
import deviceinfo
from time import time

#wksp = '../device/v1.3/'


#### Step1: OpenCL preparation

In [3]:
## Devices and compute context
platforms = cl.get_platforms()
context = cl.Context(
        dev_type=cl.device_type.ALL,
        properties=[(cl.context_properties.PLATFORM, platforms[0])])
device = platforms[0].get_devices()

# Create a command queue
queue = cl.CommandQueue(context)

context

<pyopencl.Context at 0x-46583a40 on <pyopencl.Device 'de10_nano_sharedonly : Cyclone V SoC Development Kit' on 'Intel(R) FPGA SDK for OpenCL(TM)' at 0x-5b09c690>>

#### Step 2: creat kernels
Creat & build program

Creat kernels

#### OpenCL kernel: conv1x1_NDRange.cl

conv2d1x1: 2-D 1x1 convolution. kernel size 1, stride 1  

```C
//1x1 convolution layer
//output one feature map per kernel
__kernel void conv2d1x1(
	const int input_channels, const int input_size,
	__global float * restrict input_im,
	__global const float4* restrict filter_weight,
	__global const float* restrict filter_bias,
	__global float *restrict output_im)
{
	// Adding restrict keyword
    int filter_index = get_global_id(0); // 0 - (output_channels - 1)
	int i = get_global_id(1);

	filter_weight += filter_index * input_channels;

	float bias = filter_bias[filter_index];
	
	output_im += filter_index * input_size * input_size;//start_channel is for 1x1 feature map in fire layer

	//loop over output feature map
	//for(int i = 0; i < input_size; i++)
	{
		for(int j = 0; j < input_size; j++)
		{
			float tmp = bias;
			int loc = i * input_size + j;

			for(int k = 0; k < input_channels; k++)
			{
				//float8 weight = filter_weight[k];
				//float8 feature;
				tmp += input_im[((k << 2) + 0) * input_size * input_size + loc] * filter_weight[k].s0
				     + input_im[((k << 2) + 1) * input_size * input_size + loc] * filter_weight[k].s1
					 + input_im[((k << 2) + 2) * input_size * input_size + loc] * filter_weight[k].s2
					 + input_im[((k << 2) + 3) * input_size * input_size + loc] * filter_weight[k].s3;
			}
			//add relu after conv
			output_im[i * input_size + j] = (tmp > 0.0) ? tmp : 0.0;
		}
	}
}

```
#### OpenCL kernel: conv1x1_simple_task.cl

conv2d1x1: 2-D 1x1 convolution. kernel size 1, stride 1  

```C
//1x1 convolution layer as a single kernel
//output one feature map per kernel
__kernel void conv2d1x1(
	const int input_channels, 
    const int input_size,
    const int filter_size,
	__global float * restrict input_im,
	__global const float4* restrict filter_weight,
	__global const float* restrict filter_bias,
	__global float *restrict output_im)
{
	// Adding restrict keyword
    //loop over filters
	for(int f_i = 0; f_i < filter_size; f_i++)
	{
        //filter_weight += f_i * input_channels;

        float bias = filter_bias[f_i];
	
        // output_im += input_size * input_size;//start_channel is for 1x1 feature map in fire layer
	
		for(int ij = 0; ij < (input_size * input_size); ij++)
		{
			float tmp = bias;
			// int loc = i * input_size + j; // this is equal to ij

			for(int k = 0; k < input_channels; k++)
			{
				//float8 weight = filter_weight[k];
				//float8 feature;
				/*tmp += input_im[((k << 2) + 0) * input_size * input_size + ij] * filter_weight[k + f_i * input_channels].s0
				     + input_im[((k << 2) + 1) * input_size * input_size + ij] * filter_weight[k + f_i * input_channels].s1
					 + input_im[((k << 2) + 2) * input_size * input_size + ij] * filter_weight[k + f_i * input_channels].s2
					 + input_im[((k << 2) + 3) * input_size * input_size + ij] * filter_weight[k + f_i * input_channels].s3;*/
                tmp += input_im[((k << 2) + 0) * input_size * input_size + ij] * filter_weight[k].s0
				     + input_im[((k << 2) + 1) * input_size * input_size + ij] * filter_weight[k].s1
					 + input_im[((k << 2) + 2) * input_size * input_size + ij] * filter_weight[k].s2
					 + input_im[((k << 2) + 3) * input_size * input_size + ij] * filter_weight[k].s3;
			}
			//add relu after conv
			//output_im[ij + (input_size * input_size * f_i)] = (tmp > 0.0) ? tmp : 0.0;
            output_im[ij] = (tmp > 0.0) ? tmp : 0.0;
		}
        filter_weight += input_channels;	
        output_im += input_size * input_size;//start_channel is for 1x1 feature map in fire layer
	
	}
}
```

Run OpenCL implement  

In [39]:
tamanyo=56 #input_size
canales_iniciales=64 #input_channels
canales_contraidos=16 #filter_size
canales_finales = canales_iniciales

acumulado_pytorch=0

imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32)
#imagen = np.ones((1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32)

weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)

#weights1=np.ones((canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
#bias1=np.ones((canales_contraidos,)).astype(np.float32)

squeeze_activation = nn.ReLU(inplace=True)

tic=pc()
squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida1_a_numpy=salida1_activation.detach().numpy()

toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch

####### OPENCL COMPARISON #######


In [120]:
# NDRANGE

wksp = ''

file_dir = wksp + 'conv1x1_NDRange.aocx'
#file_dir = wksp + 'conv1x1_NDRange_locmem.aocx'

kernelSource = open(file_dir, mode='rb').read()
program = cl.Program(context, device, [kernelSource]).build()

conv1x1 = program.conv2d1x1
conv1x1.set_scalar_arg_dtypes([np.int32, np.int32, None, None, None, None])
#conv1x1.set_scalar_arg_dtypes([np.int32, np.int32, None, None, None, None, None])
#loc_mem = cl.LocalMemory(np.dtype(np.float32).itemsize * 250*4) # Creating a local memory buffer indicating the size in bytes

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample1 = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1

d_fire1_squeeze_weight1 = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias1 = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
d_result_fire1_squeeze1 = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)


In [124]:
tic2 = pc()

conv1x1(queue,(canales_contraidos, tamanyo), None, np.int32(canales_iniciales/4), tamanyo, d_sample1, d_fire1_squeeze_weight1, d_fire1_squeeze_bias1, d_result_fire1_squeeze1)
#conv1x1(queue,(canales_contraidos, tamanyo), None, np.int32(canales_iniciales/4), tamanyo, loc_mem, d_sample1, d_fire1_squeeze_weight1, d_fire1_squeeze_bias1, d_result_fire1_squeeze1)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_squeeze, d_result_fire1_squeeze1)

queue.finish()

veamos = h_result_fire1_squeeze.reshape(-1,tamanyo,tamanyo)

rtime = pc() - tic2


In [43]:
# Simple task V0

wksp = ''

file_dir = wksp + 'conv1x1_simple_task.aocx'

kernelSource = open(file_dir, mode='rb').read()
program = cl.Program(context, device, [kernelSource]).build()

conv1x1 = program.conv2d1x1
conv1x1.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None, None, None])

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)


In [45]:
tic3 = pc()

conv1x1(queue,(1,), None, np.int32(canales_iniciales/4), tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_squeeze, d_result_fire1_squeeze)

queue.finish()

veamos1 = h_result_fire1_squeeze.reshape(-1,tamanyo,tamanyo)

rtime1 = pc() - tic3

In [46]:
# Simple task V1

wksp = ''

file_dir = wksp + 'conv1x1_simple_task_V1.aocx'

kernelSource = open(file_dir, mode='rb').read()
program = cl.Program(context, device, [kernelSource]).build()

conv1x1 = program.conv2d1x1
conv1x1.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None, None, None])

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)


In [48]:
tic4 = pc()

conv1x1(queue,(1,), None, np.int32(canales_iniciales/4), tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)

queue.finish()

cl.enqueue_copy(queue, h_result_fire1_squeeze, d_result_fire1_squeeze)

queue.finish()

veamos2 = h_result_fire1_squeeze.reshape(-1,tamanyo,tamanyo)

rtime2 = pc() - tic4

In [131]:
# Simple task V2 -> ahora SR

wksp = ''

#file_dir = wksp + 'conv1x1_simple_task_V2_nounroll.aocx'
#file_dir = wksp + 'conv1x1_simple_task_V2.aocx'
#file_dir = wksp + 'conv1x1_simple_task_Nofloat4.aocx'
#file_dir = wksp + 'conv1x1_simple_task_Nofloat4_a.aocx'
#file_dir = wksp + 'conv1x1_simple_task_Nofloat4_aV2.aocx' ## corrigiendo
#file_dir = wksp + 'conv1x1_simple_task_Nofloat4_b.aocx'
#file_dir = wksp + 'conv1x1_simple_task_float8SR.aocx'
file_dir = wksp + 'conv1x1_simple_task_V5.aocx'

kernelSource = open(file_dir, mode='rb').read()
program = cl.Program(context, device, [kernelSource]).build()

conv1x1 = program.conv2d1x1
conv1x1.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None, None, None])
#conv1x1.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, None, None, None, None, None])

#shift_reg = cl.LocalMemory(np.dtype(np.float32).itemsize * 60) # Creating a local memory buffer indicating the size in bytes

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)


In [133]:
tic5 = pc()

#conv1x1(queue,(1,), None, np.int32(canales_iniciales/4), tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)
#conv1x1(queue,(1,), None, np.int32(canales_iniciales/4), tamanyo, canales_contraidos, shift_reg, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)
conv1x1(queue,(1,), None, np.int32(canales_iniciales), tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)
#conv1x1(queue,(1,), None, np.int32(canales_iniciales/8), tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)
queue.finish()

cl.enqueue_copy(queue, h_result_fire1_squeeze, d_result_fire1_squeeze)

queue.finish()

veamos3 = h_result_fire1_squeeze.reshape(-1,tamanyo,tamanyo)

rtime3 = pc() - tic5

In [125]:
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con opencl (NDRANGE)=",rtime)
print ("tiempo en segundos con opencl (Simple Task V0)=",rtime1)
print ("tiempo en segundos con opencl (Simple Task V1)=",rtime2)
print ("tiempo en segundos con opencl (Simple Task V2)=",rtime3)

comparativa1=np.allclose(salida1_a_numpy, veamos,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(salida1_a_numpy, veamos1,rtol=1e-01, atol=1e-01)
comparativa3=np.allclose(salida1_a_numpy, veamos2,rtol=1e-01, atol=1e-01)
comparativa4=np.allclose(salida1_a_numpy, veamos3,rtol=1e-01, atol=1e-01)

print("comparativa (pytorch == NDRange): ",comparativa1)
print("comparativa (pytorch == Simple Task V0): ",comparativa2)
print("comparativa (pytorch == Simple Task V1): ",comparativa3)
print("comparativa (pytorch == Simple Task V2): ",comparativa4)

tiempo en segundos con pytorch=  0.03568267700029537
tiempo en segundos con opencl (NDRANGE)= 0.0156927650000398
tiempo en segundos con opencl (Simple Task V0)= 0.09479616800001622
tiempo en segundos con opencl (Simple Task V1)= 0.060706462999974065
tiempo en segundos con opencl (Simple Task V2)= 0.03159019799977614
comparativa (pytorch == NDRange):  True
comparativa (pytorch == Simple Task V0):  False
comparativa (pytorch == Simple Task V1):  False
comparativa (pytorch == Simple Task V2):  True


In [134]:
tamanyo=56 #input_size
canales_iniciales=64 #input_channels
canales_contraidos=16 #filter_size
canales_finales = canales_iniciales

count = 100

acumulado_pytorch=0
acumulado_kernel=0
comparativa4 = True

for i in range(count):
    imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32)
    #imagen = np.ones((1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32)

    weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
    bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)

    #weights1=np.ones((canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
    #bias1=np.ones((canales_contraidos,)).astype(np.float32)

    squeeze_activation = nn.ReLU(inplace=True)

    tic=pc()
    squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
    squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
    squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

    imagen1  = torch.from_numpy(imagen).float()

    salida1=squeeze1(imagen1)
    salida1_activation=squeeze_activation(salida1)

    salida1_a_numpy=salida1_activation.detach().numpy()

    toc=pc()
    acumulado_pytorch=toc-tic+acumulado_pytorch
    
    ####### OPENCL COMPARISON #######
    
    h_sample = imagen.reshape(-1).astype(np.float32)
    d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

    fire1_squeeze_weight = weights1.reshape(-1)
    fire1_squeeze_bias = bias1

    d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
    d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)

    h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
    d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
    
    tic5 = pc()

    conv1x1(queue,(1,), None, np.int32(canales_iniciales), tamanyo, canales_contraidos, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)
    
    queue.finish()

    cl.enqueue_copy(queue, h_result_fire1_squeeze, d_result_fire1_squeeze)

    queue.finish()

    veamos3 = h_result_fire1_squeeze.reshape(-1,tamanyo,tamanyo)
    
    toc5 = pc()

    acumulado_kernel = toc5 - tic5 + acumulado_kernel
    
    comparativa4 &= np.allclose(salida1_a_numpy, veamos3,rtol=1e-01, atol=1e-01)
    
print ("tiempo en segundos con pytorch= ", acumulado_pytorch/count)
print ("tiempo en segundos con opencl (Simple Task V2)=",acumulado_kernel/count)

print("comparativa (pytorch == Simple Task V2): ",comparativa4)

tiempo en segundos con pytorch=  0.011025258960021347
tiempo en segundos con opencl (Simple Task V2)= 0.02951016586998776
comparativa (pytorch == Simple Task V2):  True


In [130]:
tamanyo=56 #input_size
canales_iniciales=64 #input_channels
canales_contraidos=16 #filter_size
canales_finales = canales_iniciales

count = 100

acumulado_pytorch=0
acumulado_kernel=0
comparativa4 = True

for i in range(count):
    imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32)
    #imagen = np.ones((1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32)

    weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
    bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)

    #weights1=np.ones((canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
    #bias1=np.ones((canales_contraidos,)).astype(np.float32)

    squeeze_activation = nn.ReLU(inplace=True)

    tic=pc()
    squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
    squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
    squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

    imagen1  = torch.from_numpy(imagen).float()

    salida1=squeeze1(imagen1)
    salida1_activation=squeeze_activation(salida1)

    salida1_a_numpy=salida1_activation.detach().numpy()

    toc=pc()
    acumulado_pytorch=toc-tic+acumulado_pytorch
    
    ####### OPENCL COMPARISON #######
    
    h_sample = imagen.reshape(-1).astype(np.float32)
    d_sample1 = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

    fire1_squeeze_weight = weights1.reshape(-1)
    fire1_squeeze_bias = bias1

    d_fire1_squeeze_weight1 = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
    d_fire1_squeeze_bias1 = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)

    h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
    d_result_fire1_squeeze1 = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)

    
    tic5 = pc()

    conv1x1(queue,(canales_contraidos, tamanyo), None, np.int32(canales_iniciales/4), tamanyo, d_sample1, d_fire1_squeeze_weight1, d_fire1_squeeze_bias1, d_result_fire1_squeeze1)
    
    queue.finish()

    cl.enqueue_copy(queue, h_result_fire1_squeeze, d_result_fire1_squeeze1)

    queue.finish()

    veamos3 = h_result_fire1_squeeze.reshape(-1,tamanyo,tamanyo)
    
    toc5 = pc()

    acumulado_kernel = toc5 - tic5 + acumulado_kernel
    
    comparativa4 &= np.allclose(salida1_a_numpy, veamos3,rtol=1e-01, atol=1e-01)
    if comparativa4 == False:
        print(i)
    
print ("tiempo en segundos con pytorch= ", acumulado_pytorch/count)
print ("tiempo en segundos con opencl (Simple Task V2)=",acumulado_kernel/count)

print("comparativa (pytorch == NDRange): ",comparativa4)

tiempo en segundos con pytorch=  0.011102727289980976
tiempo en segundos con opencl (Simple Task V2)= 0.011684067709988994
comparativa (pytorch == NDRange):  True


# Simple Task no mejora

In [98]:
print("pytorch", veamos.shape, salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[4][0])
print("NDRange", veamos[4][0])
print("Simple task", veamos3[1][0])
print(imagen.shape, np.allclose(imagen, np.ones((1,canales_iniciales, tamanyo, tamanyo)).astype(np.float32), rtol=1e-01, atol=1e-01))
#print(weights1[6])
# print(fire1_squeeze_weight)
#print(bias1[6])
# print(imagen1_obtenida)
# print(veamos2)
print(np.allclose(salida1_a_numpy, veamos2,rtol=1e-01, atol=1e-01))

pytorch (16, 55, 55) [1406. 1295. 1366. 1159. 1414. 1111. 1045. 1124. 1368. 1236. 1299. 1147.
 1411. 1271. 1215. 1376. 1371. 1349. 1291. 1283. 1322. 1329. 1138. 1502.
 1298. 1426. 1251. 1173. 1412. 1224. 1362. 1453. 1360. 1165. 1344. 1315.
 1541. 1497. 1348. 1231. 1458. 1204. 1533. 1115. 1596. 1224. 1224. 1312.
 1282. 1289. 1058. 1276. 1325. 1326. 1349.]
NDRange [1406. 1295. 1366. 1159. 1414. 1111. 1045. 1124. 1368. 1236. 1299. 1147.
 1411. 1271. 1215. 1376. 1371. 1349. 1291. 1283. 1322. 1329. 1138. 1502.
 1298. 1426. 1251. 1173. 1412. 1224. 1362. 1453. 1360. 1165. 1344. 1315.
 1541. 1497. 1348. 1231. 1458. 1204. 1533. 1115. 1596. 1224. 1224. 1312.
 1282. 1289. 1058. 1276. 1325. 1326. 1349.]
Simple task [713. 735. 686. 755. 573. 538. 612. 709. 705. 548. 647. 679. 581. 749.
 648. 589. 665. 748. 605. 755. 640. 679. 513. 625. 648. 649. 712. 825.
 793. 688. 610. 835. 638. 709. 796. 743. 731. 860. 671. 681. 655. 559.
 808. 642. 674. 629. 603. 798. 721. 599. 465. 604. 882. 637. 844.]
(1, 64,

In [128]:
for i in range(16):
    for j in range(55):
        for k in range(55):
            if (abs(salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k] - veamos3[i][j][k])) > 1e-01:
                print("i:", i, "j:", j, "k:", k, salida1_a_numpy.reshape(-1,tamanyo,tamanyo)[i][j][k], veamos3[i][j][k])

i: 0 j: 0 k: 0 1382.0 0.0
i: 0 j: 0 k: 1 1324.0 0.0
i: 0 j: 0 k: 2 1480.0 0.0
i: 0 j: 0 k: 3 1258.0 0.0
i: 0 j: 0 k: 4 1533.0 0.0
i: 0 j: 0 k: 5 1470.0 0.0
i: 0 j: 0 k: 6 1358.0 0.0
i: 0 j: 0 k: 7 1425.0 0.0
i: 0 j: 0 k: 8 1312.0 0.0
i: 0 j: 0 k: 9 1460.0 0.0
i: 0 j: 0 k: 10 1254.0 0.0
i: 0 j: 0 k: 11 1687.0 0.0
i: 0 j: 0 k: 12 1415.0 0.0
i: 0 j: 0 k: 13 1791.0 0.0
i: 0 j: 0 k: 14 1279.0 0.0
i: 0 j: 0 k: 15 1592.0 0.0
i: 0 j: 0 k: 16 1502.0 0.0
i: 0 j: 0 k: 17 1446.0 0.0
i: 0 j: 0 k: 18 1451.0 0.0
i: 0 j: 0 k: 19 1346.0 0.0
i: 0 j: 0 k: 20 1106.0 0.0
i: 0 j: 0 k: 21 1486.0 0.0
i: 0 j: 0 k: 22 1422.0 0.0
i: 0 j: 0 k: 23 1397.0 0.0
i: 0 j: 0 k: 24 1365.0 0.0
i: 0 j: 0 k: 25 1304.0 0.0
i: 0 j: 0 k: 26 1549.0 0.0
i: 0 j: 0 k: 27 1561.0 0.0
i: 0 j: 0 k: 28 1334.0 0.0
i: 0 j: 0 k: 29 1362.0 0.0
i: 0 j: 0 k: 30 1382.0 0.0
i: 0 j: 0 k: 31 1451.0 0.0
i: 0 j: 0 k: 32 1263.0 0.0
i: 0 j: 0 k: 33 1430.0 0.0
i: 0 j: 0 k: 34 1184.0 0.0
i: 0 j: 0 k: 35 1486.0 0.0
i: 0 j: 0 k: 36 1323.0 0.0
i: 0 j: 0 k

i: 0 j: 8 k: 7 1315.0 0.0
i: 0 j: 8 k: 8 1427.0 0.0
i: 0 j: 8 k: 9 1470.0 0.0
i: 0 j: 8 k: 10 1524.0 0.0
i: 0 j: 8 k: 11 1617.0 0.0
i: 0 j: 8 k: 12 1336.0 0.0
i: 0 j: 8 k: 13 1305.0 0.0
i: 0 j: 8 k: 14 1388.0 0.0
i: 0 j: 8 k: 15 1523.0 0.0
i: 0 j: 8 k: 16 1390.0 0.0
i: 0 j: 8 k: 17 1388.0 0.0
i: 0 j: 8 k: 18 1198.0 0.0
i: 0 j: 8 k: 19 1586.0 0.0
i: 0 j: 8 k: 20 1517.0 0.0
i: 0 j: 8 k: 21 1353.0 0.0
i: 0 j: 8 k: 22 1617.0 0.0
i: 0 j: 8 k: 23 1333.0 0.0
i: 0 j: 8 k: 24 1206.0 0.0
i: 0 j: 8 k: 25 1422.0 0.0
i: 0 j: 8 k: 26 1327.0 0.0
i: 0 j: 8 k: 27 1375.0 0.0
i: 0 j: 8 k: 28 1225.0 0.0
i: 0 j: 8 k: 29 1422.0 0.0
i: 0 j: 8 k: 30 1486.0 0.0
i: 0 j: 8 k: 31 1507.0 0.0
i: 0 j: 8 k: 32 1494.0 0.0
i: 0 j: 8 k: 33 1400.0 0.0
i: 0 j: 8 k: 34 1293.0 0.0
i: 0 j: 8 k: 35 1538.0 0.0
i: 0 j: 8 k: 36 1082.0 0.0
i: 0 j: 8 k: 37 1313.0 0.0
i: 0 j: 8 k: 38 1456.0 0.0
i: 0 j: 8 k: 39 1506.0 0.0
i: 0 j: 8 k: 40 1273.0 0.0
i: 0 j: 8 k: 41 1486.0 0.0
i: 0 j: 8 k: 42 1275.0 0.0
i: 0 j: 8 k: 43 1380.0 0.0
i: 0

i: 0 j: 15 k: 40 1462.0 0.0
i: 0 j: 15 k: 41 1501.0 0.0
i: 0 j: 15 k: 42 1435.0 0.0
i: 0 j: 15 k: 43 1374.0 0.0
i: 0 j: 15 k: 44 1502.0 0.0
i: 0 j: 15 k: 45 1156.0 0.0
i: 0 j: 15 k: 46 1423.0 0.0
i: 0 j: 15 k: 47 1269.0 0.0
i: 0 j: 15 k: 48 1305.0 0.0
i: 0 j: 15 k: 49 1359.0 0.0
i: 0 j: 15 k: 50 1275.0 0.0
i: 0 j: 15 k: 51 1382.0 0.0
i: 0 j: 15 k: 52 1484.0 0.0
i: 0 j: 15 k: 53 1466.0 0.0
i: 0 j: 15 k: 54 1265.0 0.0
i: 0 j: 16 k: 0 1323.0 0.0
i: 0 j: 16 k: 1 1340.0 0.0
i: 0 j: 16 k: 2 1433.0 0.0
i: 0 j: 16 k: 3 1349.0 0.0
i: 0 j: 16 k: 4 1531.0 0.0
i: 0 j: 16 k: 5 1384.0 0.0
i: 0 j: 16 k: 6 1319.0 0.0
i: 0 j: 16 k: 7 1439.0 0.0
i: 0 j: 16 k: 8 1350.0 0.0
i: 0 j: 16 k: 9 1428.0 0.0
i: 0 j: 16 k: 10 1034.0 0.0
i: 0 j: 16 k: 11 1412.0 0.0
i: 0 j: 16 k: 12 1551.0 0.0
i: 0 j: 16 k: 13 1339.0 0.0
i: 0 j: 16 k: 14 1479.0 0.0
i: 0 j: 16 k: 15 1421.0 0.0
i: 0 j: 16 k: 16 1242.0 0.0
i: 0 j: 16 k: 17 1419.0 0.0
i: 0 j: 16 k: 18 1193.0 0.0
i: 0 j: 16 k: 19 1424.0 0.0
i: 0 j: 16 k: 20 1686.0 0.0
i:

i: 0 j: 23 k: 37 1295.0 0.0
i: 0 j: 23 k: 38 1499.0 0.0
i: 0 j: 23 k: 39 1566.0 0.0
i: 0 j: 23 k: 40 1163.0 0.0
i: 0 j: 23 k: 41 1414.0 0.0
i: 0 j: 23 k: 42 1293.0 0.0
i: 0 j: 23 k: 43 1303.0 0.0
i: 0 j: 23 k: 44 1306.0 0.0
i: 0 j: 23 k: 45 1289.0 0.0
i: 0 j: 23 k: 46 1363.0 0.0
i: 0 j: 23 k: 47 1243.0 0.0
i: 0 j: 23 k: 48 1385.0 0.0
i: 0 j: 23 k: 49 1555.0 0.0
i: 0 j: 23 k: 50 1456.0 0.0
i: 0 j: 23 k: 51 1322.0 0.0
i: 0 j: 23 k: 52 1693.0 0.0
i: 0 j: 23 k: 53 1595.0 0.0
i: 0 j: 23 k: 54 1255.0 0.0
i: 0 j: 24 k: 0 1480.0 0.0
i: 0 j: 24 k: 1 1444.0 0.0
i: 0 j: 24 k: 2 1309.0 0.0
i: 0 j: 24 k: 3 1314.0 0.0
i: 0 j: 24 k: 4 1431.0 0.0
i: 0 j: 24 k: 5 1265.0 0.0
i: 0 j: 24 k: 6 1569.0 0.0
i: 0 j: 24 k: 7 1281.0 0.0
i: 0 j: 24 k: 8 1367.0 0.0
i: 0 j: 24 k: 9 1398.0 0.0
i: 0 j: 24 k: 10 1365.0 0.0
i: 0 j: 24 k: 11 1534.0 0.0
i: 0 j: 24 k: 12 1411.0 0.0
i: 0 j: 24 k: 13 1422.0 0.0
i: 0 j: 24 k: 14 1446.0 0.0
i: 0 j: 24 k: 15 1341.0 0.0
i: 0 j: 24 k: 16 1231.0 0.0
i: 0 j: 24 k: 17 1282.0 0.0
i:

i: 0 j: 32 k: 16 1466.0 0.0
i: 0 j: 32 k: 17 1282.0 0.0
i: 0 j: 32 k: 18 1262.0 0.0
i: 0 j: 32 k: 19 1234.0 0.0
i: 0 j: 32 k: 20 1477.0 0.0
i: 0 j: 32 k: 21 1250.0 0.0
i: 0 j: 32 k: 22 1665.0 0.0
i: 0 j: 32 k: 23 1305.0 0.0
i: 0 j: 32 k: 24 1603.0 0.0
i: 0 j: 32 k: 25 1396.0 0.0
i: 0 j: 32 k: 26 1274.0 0.0
i: 0 j: 32 k: 27 1162.0 0.0
i: 0 j: 32 k: 28 1602.0 0.0
i: 0 j: 32 k: 29 1533.0 0.0
i: 0 j: 32 k: 30 1323.0 0.0
i: 0 j: 32 k: 31 1341.0 0.0
i: 0 j: 32 k: 32 1488.0 0.0
i: 0 j: 32 k: 33 1703.0 0.0
i: 0 j: 32 k: 34 1313.0 0.0
i: 0 j: 32 k: 35 1353.0 0.0
i: 0 j: 32 k: 36 1215.0 0.0
i: 0 j: 32 k: 37 1662.0 0.0
i: 0 j: 32 k: 38 1329.0 0.0
i: 0 j: 32 k: 39 1318.0 0.0
i: 0 j: 32 k: 40 1243.0 0.0
i: 0 j: 32 k: 41 1173.0 0.0
i: 0 j: 32 k: 42 1715.0 0.0
i: 0 j: 32 k: 43 1536.0 0.0
i: 0 j: 32 k: 44 1422.0 0.0
i: 0 j: 32 k: 45 1604.0 0.0
i: 0 j: 32 k: 46 1205.0 0.0
i: 0 j: 32 k: 47 1369.0 0.0
i: 0 j: 32 k: 48 1263.0 0.0
i: 0 j: 32 k: 49 1492.0 0.0
i: 0 j: 32 k: 50 1341.0 0.0
i: 0 j: 32 k: 51 122

...

i: 15 j: 14 k: 32 1429.0 0.0
i: 15 j: 14 k: 33 1369.0 0.0
i: 15 j: 14 k: 34 1180.0 0.0
i: 15 j: 14 k: 35 1104.0 0.0
i: 15 j: 14 k: 36 1561.0 0.0
i: 15 j: 14 k: 37 1167.0 0.0
i: 15 j: 14 k: 38 1306.0 0.0
i: 15 j: 14 k: 39 1167.0 0.0
i: 15 j: 14 k: 40 1166.0 0.0
i: 15 j: 14 k: 41 1238.0 0.0
i: 15 j: 14 k: 42 1183.0 0.0
i: 15 j: 14 k: 43 1191.0 0.0
i: 15 j: 14 k: 44 1333.0 0.0
i: 15 j: 14 k: 45 1180.0 0.0
i: 15 j: 14 k: 46 1355.0 0.0
i: 15 j: 14 k: 47 1110.0 0.0
i: 15 j: 14 k: 48 1288.0 0.0
i: 15 j: 14 k: 49 1193.0 0.0
i: 15 j: 14 k: 50 1167.0 0.0
i: 15 j: 14 k: 51 1285.0 0.0
i: 15 j: 14 k: 52 1486.0 0.0
i: 15 j: 14 k: 53 1051.0 0.0
i: 15 j: 14 k: 54 1320.0 0.0
i: 15 j: 15 k: 0 1313.0 0.0
i: 15 j: 15 k: 1 1217.0 0.0
i: 15 j: 15 k: 2 1089.0 0.0
i: 15 j: 15 k: 3 1204.0 0.0
i: 15 j: 15 k: 4 1303.0 0.0
i: 15 j: 15 k: 5 1113.0 0.0
i: 15 j: 15 k: 6 1302.0 0.0
i: 15 j: 15 k: 7 1011.0 0.0
i: 15 j: 15 k: 8 1321.0 0.0
i: 15 j: 15 k: 9 1386.0 0.0
i: 15 j: 15 k: 10 1339.0 0.0
i: 15 j: 15 k: 11 1149.0

i: 15 j: 23 k: 8 1211.0 0.0
i: 15 j: 23 k: 9 1207.0 0.0
i: 15 j: 23 k: 10 1215.0 0.0
i: 15 j: 23 k: 11 1234.0 0.0
i: 15 j: 23 k: 12 1194.0 0.0
i: 15 j: 23 k: 13 1137.0 0.0
i: 15 j: 23 k: 14 1452.0 0.0
i: 15 j: 23 k: 15 1189.0 0.0
i: 15 j: 23 k: 16 1439.0 0.0
i: 15 j: 23 k: 17 948.0 0.0
i: 15 j: 23 k: 18 1223.0 0.0
i: 15 j: 23 k: 19 1226.0 0.0
i: 15 j: 23 k: 20 1497.0 0.0
i: 15 j: 23 k: 21 1161.0 0.0
i: 15 j: 23 k: 22 1268.0 0.0
i: 15 j: 23 k: 23 1246.0 0.0
i: 15 j: 23 k: 24 1341.0 0.0
i: 15 j: 23 k: 25 1182.0 0.0
i: 15 j: 23 k: 26 1341.0 0.0
i: 15 j: 23 k: 27 1483.0 0.0
i: 15 j: 23 k: 28 1451.0 0.0
i: 15 j: 23 k: 29 1186.0 0.0
i: 15 j: 23 k: 30 1282.0 0.0
i: 15 j: 23 k: 31 1243.0 0.0
i: 15 j: 23 k: 32 996.0 0.0
i: 15 j: 23 k: 33 1133.0 0.0
i: 15 j: 23 k: 34 1267.0 0.0
i: 15 j: 23 k: 35 1297.0 0.0
i: 15 j: 23 k: 36 1245.0 0.0
i: 15 j: 23 k: 37 1044.0 0.0
i: 15 j: 23 k: 38 1416.0 0.0
i: 15 j: 23 k: 39 1358.0 0.0
i: 15 j: 23 k: 40 1062.0 0.0
i: 15 j: 23 k: 41 1319.0 0.0
i: 15 j: 23 k: 42 

i: 15 j: 31 k: 40 1162.0 0.0
i: 15 j: 31 k: 41 1133.0 0.0
i: 15 j: 31 k: 42 1283.0 0.0
i: 15 j: 31 k: 43 1288.0 0.0
i: 15 j: 31 k: 44 1326.0 0.0
i: 15 j: 31 k: 45 1261.0 0.0
i: 15 j: 31 k: 46 1156.0 0.0
i: 15 j: 31 k: 47 1109.0 0.0
i: 15 j: 31 k: 48 1404.0 0.0
i: 15 j: 31 k: 49 1277.0 0.0
i: 15 j: 31 k: 50 1278.0 0.0
i: 15 j: 31 k: 51 1433.0 0.0
i: 15 j: 31 k: 52 935.0 0.0
i: 15 j: 31 k: 53 1058.0 0.0
i: 15 j: 31 k: 54 1294.0 0.0
i: 15 j: 32 k: 0 1163.0 0.0
i: 15 j: 32 k: 1 1028.0 0.0
i: 15 j: 32 k: 2 1319.0 0.0
i: 15 j: 32 k: 3 1305.0 0.0
i: 15 j: 32 k: 4 997.0 0.0
i: 15 j: 32 k: 5 989.0 0.0
i: 15 j: 32 k: 6 1239.0 0.0
i: 15 j: 32 k: 7 1487.0 0.0
i: 15 j: 32 k: 8 1166.0 0.0
i: 15 j: 32 k: 9 1094.0 0.0
i: 15 j: 32 k: 10 1422.0 0.0
i: 15 j: 32 k: 11 1410.0 0.0
i: 15 j: 32 k: 12 1270.0 0.0
i: 15 j: 32 k: 13 1078.0 0.0
i: 15 j: 32 k: 14 1210.0 0.0
i: 15 j: 32 k: 15 1269.0 0.0
i: 15 j: 32 k: 16 1417.0 0.0
i: 15 j: 32 k: 17 1384.0 0.0
i: 15 j: 32 k: 18 1158.0 0.0
i: 15 j: 32 k: 19 1228.0 0.

i: 15 j: 40 k: 17 1345.0 0.0
i: 15 j: 40 k: 18 1380.0 0.0
i: 15 j: 40 k: 19 1270.0 0.0
i: 15 j: 40 k: 20 1461.0 0.0
i: 15 j: 40 k: 21 1039.0 0.0
i: 15 j: 40 k: 22 1239.0 0.0
i: 15 j: 40 k: 23 1109.0 0.0
i: 15 j: 40 k: 24 1160.0 0.0
i: 15 j: 40 k: 25 1449.0 0.0
i: 15 j: 40 k: 26 1210.0 0.0
i: 15 j: 40 k: 27 1178.0 0.0
i: 15 j: 40 k: 28 1113.0 0.0
i: 15 j: 40 k: 29 1354.0 0.0
i: 15 j: 40 k: 30 1111.0 0.0
i: 15 j: 40 k: 31 1183.0 0.0
i: 15 j: 40 k: 32 1127.0 0.0
i: 15 j: 40 k: 33 1296.0 0.0
i: 15 j: 40 k: 34 1273.0 0.0
i: 15 j: 40 k: 35 1213.0 0.0
i: 15 j: 40 k: 36 1243.0 0.0
i: 15 j: 40 k: 37 1140.0 0.0
i: 15 j: 40 k: 38 1339.0 0.0
i: 15 j: 40 k: 39 1396.0 0.0
i: 15 j: 40 k: 40 1393.0 0.0
i: 15 j: 40 k: 41 1067.0 0.0
i: 15 j: 40 k: 42 1253.0 0.0
i: 15 j: 40 k: 43 1292.0 0.0
i: 15 j: 40 k: 44 1287.0 0.0
i: 15 j: 40 k: 45 1310.0 0.0
i: 15 j: 40 k: 46 1254.0 0.0
i: 15 j: 40 k: 47 1423.0 0.0
i: 15 j: 40 k: 48 1324.0 0.0
i: 15 j: 40 k: 49 1338.0 0.0
i: 15 j: 40 k: 50 1434.0 0.0
i: 15 j: 40 k:

i: 15 j: 48 k: 50 1089.0 0.0
i: 15 j: 48 k: 51 1320.0 0.0
i: 15 j: 48 k: 52 1232.0 0.0
i: 15 j: 48 k: 53 1286.0 0.0
i: 15 j: 48 k: 54 1235.0 0.0
i: 15 j: 49 k: 0 1294.0 0.0
i: 15 j: 49 k: 1 1396.0 0.0
i: 15 j: 49 k: 2 1222.0 0.0
i: 15 j: 49 k: 3 1179.0 0.0
i: 15 j: 49 k: 4 1414.0 0.0
i: 15 j: 49 k: 5 1213.0 0.0
i: 15 j: 49 k: 6 1136.0 0.0
i: 15 j: 49 k: 7 991.0 0.0
i: 15 j: 49 k: 8 1109.0 0.0
i: 15 j: 49 k: 9 1189.0 0.0
i: 15 j: 49 k: 10 1200.0 0.0
i: 15 j: 49 k: 11 1265.0 0.0
i: 15 j: 49 k: 12 1314.0 0.0
i: 15 j: 49 k: 13 1107.0 0.0
i: 15 j: 49 k: 14 1232.0 0.0
i: 15 j: 49 k: 15 1332.0 0.0
i: 15 j: 49 k: 16 1133.0 0.0
i: 15 j: 49 k: 17 1279.0 0.0
i: 15 j: 49 k: 18 1078.0 0.0
i: 15 j: 49 k: 19 1247.0 0.0
i: 15 j: 49 k: 20 1327.0 0.0
i: 15 j: 49 k: 21 1263.0 0.0
i: 15 j: 49 k: 22 1253.0 0.0
i: 15 j: 49 k: 23 1094.0 0.0
i: 15 j: 49 k: 24 1403.0 0.0
i: 15 j: 49 k: 25 1243.0 0.0
i: 15 j: 49 k: 26 1394.0 0.0
i: 15 j: 49 k: 27 1375.0 0.0
i: 15 j: 49 k: 28 1350.0 0.0
i: 15 j: 49 k: 29 1171.0 

In [33]:
print(torch.__version__)


1.13.0a0+git49444c3
